<a href="https://colab.research.google.com/github/asafct2003/DEEP_LEARNING/blob/main/FaceMaskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Set dataset path
dataset_path = os.path.join(path, "data")   # using kagglehub 'path'

with_mask_path = os.path.join(dataset_path, "with_mask")
without_mask_path = os.path.join(dataset_path, "without_mask")

# Get file names
with_mask_images = os.listdir(with_mask_path)[:5]       # first 5 images
without_mask_images = os.listdir(without_mask_path)[:5] # first 5 images

plt.figure(figsize=(12, 6))

# Display With Mask
for i, img_name in enumerate(with_mask_images):
    img = imread(os.path.join(with_mask_path, img_name))
    plt.subplot(2, 5, i+1)
    plt.imshow(img)
    plt.title("With Mask")
    plt.axis("off")

# Display Without Mask
for i, img_name in enumerate(without_mask_images):
    img = imread(os.path.join(without_mask_path, img_name))
    plt.subplot(2, 5, i+6)
    plt.imshow(img)
    plt.title("Without Mask")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
IMG_SIZE = 150
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.20,    # 20% for testing
    rotation_range=15,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

test_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    train_data,
    epochs=15,
    validation_data=test_data
)


In [ ]:
loss, acc = model.evaluate(test_data)
print("Test Accuracy:", acc)

In [ ]:

import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/content/OIP mask.jpg"   # your image file

# Load and preprocess
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img) / 255.0
img_expanded = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_expanded)[0][0]

# Display image
plt.imshow(img)
plt.title("Input Image")
plt.axis("off")
plt.show()

# Print prediction
if prediction < 0.5:
    print("Mask Detected 😷")
else:
    print("No Mask Detected 😐")

In [ ]:

import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/content/OIP withoutmask.jpg"   # your image file

# Load and preprocess
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img) / 255.0
img_expanded = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_expanded)[0][0]

# Display image
plt.imshow(img)
plt.title("Input Image")
plt.axis("off")
plt.show()

# Print prediction
if prediction < 0.5:
    print("Mask Detected 😷")
else:
    print("No Mask Detected 😐")